**STREAMLIT APP**

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!cp tb_vgg16_model.pth "/content/drive/MyDrive/TB_detection_project/tb_vgg16_model.pth"

In [6]:
!pip install streamlit torch torchvision pillow

In [7]:
!pip install pyngrok

In [8]:
%%writefile app.py

import streamlit as st
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

# ---------------- Page configuration ----------------
st.set_page_config(page_title="TB Detection", page_icon="🫁", layout="wide")

# ---------------- Model loading ----------------
@st.cache_resource
def load_model():
    model = models.vgg16(weights=None)
    in_features = model.classifier[-1].in_features
    model.classifier[-1] = nn.Linear(in_features, 1)   # single logit: TB vs Normal

    checkpoint = torch.load("tb_vgg16_model.pth", map_location="cpu")

    if isinstance(checkpoint, dict) and "model_state_dict" in checkpoint:
        state_dict = checkpoint["model_state_dict"]
    else:
        state_dict = checkpoint

    model.load_state_dict(state_dict)
    model.eval()
    return model

# ---------------- Preprocessing ----------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

def predict_image(model, image):
    image = image.convert("RGB")
    image = transform(image).unsqueeze(0)

    with torch.no_grad():
        logits = model(image)
        prob_tb = torch.sigmoid(logits)[0].item()
        prediction = "TB" if prob_tb >= 0.5 else "Normal"
        confidence = prob_tb if prediction == "TB" else 1.0 - prob_tb

    return prediction, confidence

# ---------------- Page selection ----------------
page = st.sidebar.selectbox("Choose a page", ["🏠 Introduction", "🔬 TB Prediction"])

# ---------------- Introduction page ----------------
if page == "🏠 Introduction":
    st.title("🫁 Tuberculosis Detection from Chest X-rays")
    st.markdown("""
    ## Welcome to TB-Detect AI

    This app uses a **VGG16 deep learning model** trained on chest X-ray images
    to detect tuberculosis (TB) vs Normal cases.
    """)

    col1, col2 = st.columns(2)
    with col1:
        st.info("✅ Upload any chest X-ray (JPG, PNG)")
        st.success("🎯 Binary classification: TB vs Normal")
    with col2:
        st.warning("⚠️ Medical disclaimer: For research/demo only. Not for clinical use.")

# ---------------- Prediction page ----------------
elif page == "🔬 TB Prediction":
    st.title("🔬 Upload Chest X-ray for TB Detection")

    uploaded_file = st.file_uploader(
        "Choose a chest X-ray image...",
        type=["png", "jpg", "jpeg"],
        help="Upload JPG/PNG chest X-ray image"
    )

    if uploaded_file is not None:
        image = Image.open(uploaded_file)
        st.image(image, caption="Uploaded Chest X-ray", use_column_width=True)

        with st.spinner("Analyzing image..."):
            model = load_model()
            prediction, confidence = predict_image(model, image)

            col1, col2, col3 = st.columns([1, 2, 1])

            with col1:
                st.markdown("**Prediction:**")
            with col2:
                if prediction == "TB":
                    st.error(f"🦠 **{prediction}**")
                else:
                    st.success(f"✅ **{prediction}**")
            with col3:
                st.markdown("**Confidence:**")
                st.metric("Confidence", f"{confidence:.1%}")

            st.markdown("---")
            st.info(f"""
            **Result Details:**
            - Prediction: **{prediction}**
            - Confidence: **{confidence:.1%}**
            - Model: **VGG16** (best performing in your training)

            *Upload another image to test more cases!*
            """)

# ---------------- Footer ----------------
st.markdown("---")
st.markdown("*Powered by VGG16 • Developed for TB Detection Research*")


Overwriting app.py


In [11]:
!pip install cloudflared
!pip install streamlit -q
!wget -q -O cloudflared.deb https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared.deb

!streamlit run /content/app.py &>/content/logs.txt &
!sleep 3
!cloudflared tunnel --url http://localhost:8501 --no-autoupdate

(Reading database ... 121717 files and directories currently installed.)
Preparing to unpack cloudflared.deb ...
Unpacking cloudflared (2025.11.1) over (2025.11.1) ...
Setting up cloudflared (2025.11.1) ...
Processing triggers for man-db (2.10.2-1) ...
2025-12-06T06:27:26Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-12-06T06:27:26Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-12-06T06:27:30Z INF +------------------------------------------